In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as px

# ==============================
# 1. LOAD KAGGLE TEXT FILE
# ==============================
df = pd.read_csv(
    "household_power_consumption.txt",
    sep=';',
    low_memory=False,
    na_values='?'
)

# ==============================
# 2. DATA PREPROCESSING
# ==============================
# Combine Date and Time into datetime
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')

# Convert power column to numeric
df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce')

# Remove missing values
df.dropna(subset=['datetime', 'Global_active_power'], inplace=True)

# Set datetime as index
df.set_index('datetime', inplace=True)

# ==============================
# 3. RESAMPLE TO HOURLY DATA
# ==============================
hourly_df = df['Global_active_power'].resample('H').mean()

hourly_df = hourly_df.reset_index()
hourly_df.columns = ['timestamp', 'kwh']

# ==============================
# 4. FEATURE ENGINEERING
# ==============================
hourly_df['hour'] = hourly_df['timestamp'].dt.hour

# Moving average smoothing (past 24 hours)
hourly_df['moving_avg'] = hourly_df['kwh'].rolling(window=24).mean()

# Remove NaN rows
hourly_df.dropna(inplace=True)

# ==============================
# 5. LINEAR REGRESSION MODEL
# ==============================
X = hourly_df[['hour']]
y = hourly_df['moving_avg']

model = LinearRegression()
model.fit(X, y)

# Predict electricity demand
hourly_df['predicted_kwh'] = model.predict(X)

# ==============================
# 6. PLOTLY DASHBOARD
# ==============================
fig = px.line(
    hourly_df.tail(168),  # last 7 days
    x='timestamp',
    y=['kwh', 'moving_avg', 'predicted_kwh'],
    labels={'value': 'Electricity Consumption (kWh)', 'variable': 'Legend'},
    title='Peak Hour Electricity Spikes (Dormitory Analysis)'
)

fig.show()


/tmp/ipython-input-2687808465.py:20: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')
/tmp/ipython-input-2687808465.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = df['Global_active_power'].resample('H').mean()


In [8]:
import plotly.express as px

fig_evening_peaks = px.bar(
    evening_peak_analysis.reset_index(),
    x='hour',
    y=['kwh', 'moving_avg', 'predicted_kwh'],
    barmode='group',
    labels={'value': 'Electricity Consumption (kWh)', 'variable': 'Metric', 'hour': 'Hour of Day'},
    title='Average Electricity Consumption and Prediction During Evening Hours (18:00 - 23:00)'
)

fig_evening_peaks.update_layout(xaxis=dict(tickmode='array', tickvals=evening_peak_analysis.index))
fig_evening_peaks.show()

In [12]:
fig1 = px.line(
    hourly_df,
    x='timestamp',
    y='kwh',
    title='Hourly Electricity Consumption Trend'
)
fig1.show()


In [13]:
fig2 = px.line(
    hourly_df,
    x='timestamp',
    y=['kwh', 'moving_avg'],
    title='Effect of Moving Average Smoothing'
)
fig2.show()


In [14]:
evening_df = hourly_df[
    (hourly_df['hour'] >= 18) & (hourly_df['hour'] <= 21)
]

fig3 = px.line(
    evening_df,
    x='timestamp',
    y='kwh',
    title='Evening Peak Hour Electricity Consumption (6 PM – 9 PM)'
)
fig3.show()


In [15]:
fig4 = px.line(
    hourly_df.tail(168),
    x='timestamp',
    y=['kwh', 'predicted_kwh'],
    title='Actual vs Predicted Electricity Demand (Past Week)'
)
fig4.show()


In [16]:
hourly_avg = hourly_df.groupby('hour')['kwh'].mean().reset_index()

fig5 = px.bar(
    hourly_avg,
    x='hour',
    y='kwh',
    title='Average Electricity Consumption by Hour of Day'
)
fig5.show()


In [17]:
hourly_df['residual'] = hourly_df['kwh'] - hourly_df['predicted_kwh']

fig6 = px.scatter(
    hourly_df.tail(168),
    x='timestamp',
    y='residual',
    title='Residual Error in Electricity Demand Prediction'
)
fig6.show()
